## Домашнее задание 9. Функции и работа с данными

## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:
- оценка 2 и ниже - низкий рейтинг;
- оценка 4 и ниже - средний рейтинг;
- оценка 4.5 и 5 - высокий рейтинг.
Результат классификации запишите в столбец class.lass.

In [207]:
import pandas as pd

In [209]:
# Импортируем таблицу Рейтинга
ratings = pd.read_csv('data/DZ9/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [253]:
def movie_rating(df):
    """Возвращает значение среднего рейтинга для movieId"""
    rat_sum = df.agg({'rating':'sum'})['rating']
    rat_cou = df.agg({'rating':'count'})['rating']
    # среднее значение = сумма всех оценок / количество оценок
    return rat_sum/rat_cou

In [255]:
grouped_ratings = ratings.copy() # создадим копию таблицы Рейтинга
# Сгруппируем оценки по movieId и вычислим для каждого среднюю оценку
grouped_ratings = grouped_ratings.groupby('movieId').apply(movie_rating)
grouped_ratings.head()

C:\Users\imary\AppData\Local\Temp\ipykernel_35760\2785660372.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_ratings = grouped_ratings.groupby('movieId').apply(movie_rating)


movieId
1    3.872470
2    3.401869
3    3.161017
4    2.384615
5    3.267857
dtype: float64

In [256]:
# Импортируем таблицу Фильмов
movies = pd.read_csv('data/DZ9/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [257]:
def adding_class(df, gr):
    """Добавляет каждой строке столбец class, в котором фильму присваивается рейтинг"""
    # в списке фильмов могут находится фильмы без оценок
    try:
        movie_rate = gr[df['movieId']]
    except KeyError:
        movie_rate = 0
    # в зависимости от средней оценки каждый фильм получает словесный рейтинг
    if movie_rate == 0:
        movie_class = 'Рейтинг отсутствует'
    elif movie_rate <= 2:
        movie_class = 'Низкий рейтинг'
    elif movie_rate <= 4:
        movie_class = 'Средний рейтинг'
    elif movie_rate <= 5:
        movie_class = 'Высокий рейтинг'   
    # словесные рейтинги записываются в новый столбец class
    df['class'] = movie_class
    return df

In [258]:
# Применим функцию классификатора к таблице построчно
movies = movies.apply(adding_class, gr=grouped_ratings, axis=1)
movies.head()

,movieId,title,genres,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Средний рейтинг
1,2,Jumanji (1995),Adventure|Children|Fantasy,Средний рейтинг
2,3,Grumpier Old Men (1995),Comedy|Romance,Средний рейтинг
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Средний рейтинг
4,5,Father of the Bride Part II (1995),Comedy,Средний рейтинг


In [263]:
# Экспортируем составленную таблицу в новый csv-файл
movies.to_csv('data/DZ9/classified_movies.csv')

## Задание 2
Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую при надлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.


Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

In [269]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [294]:
# Импортируем таблицу Рейтинга
keywords = pd.read_csv('data/DZ9/keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [338]:
def geo_classificator(df, geo_data):
    """Добавляет каждой строке столбец region, в котором запросу присваивается геопринадлежность"""
    # разбиваем запрос на слова, понижаем регистр
    words = df['keyword'].lower().split(' ')
    # среди слов ищем названия городов, поддающихся классификации
    for word in words:
        for key in geo_data.keys():
            if word in geo_data[key]:
                region = key
            else:
                region = 'undefined'
    # названия регионов записываются в новый столбец region
    df['region'] = region
    return df

In [340]:
# Применим гео-классификатор к таблице Поисковых запросов
keywords = keywords.apply(geo_classificator, geo_data=geo_data, axis=1)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [342]:
# Экспортируем составленную таблицу в новый csv-файл
keywords.to_csv('data/DZ9/geo_classified_keywords.csv')